In [539]:
import pandas as pd
from datetime import timedelta
import random

In [540]:
col_numero_protocolo       = 'numero_protocolo'
col_data_abertura          = 'data_abertura_protocolo'
col_procedencia            = 'procedencia'
col_data_vencimento        = 'data_vencimento'
col_intervalo              = 'intervalo'
col_data_resposta          = 'data_resposta'
col_dias_ate_resposta      = 'resposta_menos_abertura'
col_dias_faltam_vencer     = 'dias_faltam_para_vencimento'

In [541]:
num_protocolos = 2000
numero_protocolo = list(range(1001, 1001 + num_protocolos))

# Data de hoje
hoje = pd.Timestamp('today').normalize()

# Gera a data de abertura: garante que seja no mínimo hoje.
data_abertura_protocolo = [
    max(pd.Timestamp('2025-01-20') + pd.Timedelta(days=random.randint(0, 30)), hoje)
    for _ in range(num_protocolos)
]

num_falsos = int(num_protocolos * 0.80)
num_verdadeiros = num_protocolos - num_falsos

procedencia = [False] * num_falsos + [True] * num_verdadeiros
random.shuffle(procedencia)

# Gera a data de vencimento: a partir da data de abertura mais um período aleatório entre 10 e 14 dias.
data_vencimento = [
    data_abertura_protocolo[i] + pd.Timedelta(days=random.randint(10, 14))
    for i in range(num_protocolos)
]

# Como data_abertura é garantida >= hoje, data_vencimento será sempre > hoje.
df = pd.DataFrame({
    col_numero_protocolo: numero_protocolo,
    col_data_abertura: data_abertura_protocolo,
    col_procedencia: procedencia,
    col_data_vencimento: data_vencimento
})

df

,numero_protocolo,data_abertura_protocolo,procedencia,data_vencimento
0,1001,2025-02-19,False,2025-03-01
1,1002,2025-02-19,False,2025-03-04
2,1003,2025-02-19,False,2025-03-03
3,1004,2025-02-19,True,2025-03-01
4,1005,2025-02-19,False,2025-03-01
...,...,...,...,...
1995,2996,2025-02-19,False,2025-03-04
1996,2997,2025-02-19,False,2025-03-04
1997,2998,2025-02-19,False,2025-03-02
1998,2999,2025-02-19,False,2025-03-01


In [542]:
df[col_data_abertura] = pd.to_datetime(df[col_data_abertura])
df[col_data_vencimento] = pd.to_datetime(df[col_data_vencimento])

In [543]:
hoje = pd.Timestamp('today').normalize()
hoje

Timestamp('2025-02-19 00:00:00')

In [544]:
conta_resposta = {} # qtd de vezes que cada data foi sugerida

# lista de datas disponíveis para resposta para um protocolo
def datas_disponiveis(abertura, vencimento):
    inicio = max(abertura + pd.Timedelta(days=2), hoje)
    fim = vencimento - pd.Timedelta(days=2)
    if fim < inicio:
        return []
    return [inicio + pd.Timedelta(days=i) for i in range((fim - inicio).days + 1)]

In [545]:
df[col_dias_faltam_vencer] = (df[col_data_vencimento] - hoje).dt.days

# ordenando os protocolos para que os procedentes e os com intervalos mais curtos sejam atendidos primeiro
df[col_intervalo] = df.apply(lambda row: (row[col_data_vencimento] - row[col_data_abertura]).days, axis=1)
df = df.sort_values(by=[col_procedencia, col_dias_faltam_vencer], ascending=[False, True]).reset_index(drop=True)
df

,numero_protocolo,data_abertura_protocolo,procedencia,data_vencimento,dias_faltam_para_vencimento,intervalo
0,1004,2025-02-19,True,2025-03-01,10,10
1,1031,2025-02-19,True,2025-03-01,10,10
2,1071,2025-02-19,True,2025-03-01,10,10
3,1091,2025-02-19,True,2025-03-01,10,10
4,1122,2025-02-19,True,2025-03-01,10,10
...,...,...,...,...,...,...
1995,2966,2025-02-19,False,2025-03-05,14,14
1996,2976,2025-02-19,False,2025-03-05,14,14
1997,2978,2025-02-19,False,2025-03-05,14,14
1998,2985,2025-02-19,False,2025-03-05,14,14


In [546]:
data_resposta_list = []

for idx, row in df.iterrows():
    disponiveis = datas_disponiveis(row[col_data_abertura], row[col_data_vencimento])
    if disponiveis:
        # Dicionário de frequências
        freq = {d: conta_resposta.get(d, 0) for d in disponiveis}
        # Uso mínimo atual
        min_freq = min(freq.values())
        # Pega as datas com esse uso mínimo
        candidatas = [d for d in disponiveis if freq[d] == min_freq]
        
        # Se procedência for True => escolha a data mais cedo (min)
        # Se procedência for False => escolha a data mais tarde (max)
        if row[col_procedencia]:
            data_escolhida = min(candidatas)
        else:
            data_escolhida = max(candidatas)
        
        # Atualiza o uso na data escolhida
        conta_resposta[data_escolhida] = conta_resposta.get(data_escolhida, 0) + 1
    else:
        # Se não há intervalo válido
        data_escolhida = pd.NaT
    
    # Guarda a data escolhida na lista final
    data_resposta_list.append(data_escolhida)

# Adiciona a coluna de data de resposta ao dataframe
df[col_data_resposta] = data_resposta_list

df = df.sort_values(by=col_numero_protocolo).reset_index(drop=True)

df

,numero_protocolo,data_abertura_protocolo,procedencia,data_vencimento,dias_faltam_para_vencimento,intervalo,data_resposta
0,1001,2025-02-19,False,2025-03-01,10,10,2025-02-27
1,1002,2025-02-19,False,2025-03-04,13,13,2025-03-02
2,1003,2025-02-19,False,2025-03-03,12,12,2025-03-01
3,1004,2025-02-19,True,2025-03-01,10,10,2025-02-21
4,1005,2025-02-19,False,2025-03-01,10,10,2025-02-26
...,...,...,...,...,...,...,...
1995,2996,2025-02-19,False,2025-03-04,13,13,2025-03-02
1996,2997,2025-02-19,False,2025-03-04,13,13,2025-03-01
1997,2998,2025-02-19,False,2025-03-02,11,11,2025-02-25
1998,2999,2025-02-19,False,2025-03-01,10,10,2025-02-21


In [547]:
df[col_data_resposta].value_counts().sort_index()

data_resposta
2025-02-21    181
2025-02-22    181
2025-02-23    182
2025-02-24    182
2025-02-25    182
2025-02-26    182
2025-02-27    182
2025-02-28    182
2025-03-01    182
2025-03-02    182
2025-03-03    182
Name: count, dtype: int64

In [548]:
# diferença
df[col_dias_ate_resposta] = (df[col_data_resposta] - df[col_data_abertura]).dt.days

# médias
media_true = df.loc[df[col_procedencia] == True, col_dias_ate_resposta].mean()
media_false = df.loc[df[col_procedencia] == False, col_dias_ate_resposta].mean()

print("média de dias procedencia true:", media_true)
print("média de dias procedencia false:", media_false)

média de dias procedencia true: 6.965
média de dias procedencia false: 7.014375


In [549]:
df['vencimento_menos_resposta'] = (df[col_data_vencimento] - df[col_data_resposta]).dt.days
df

,numero_protocolo,data_abertura_protocolo,procedencia,data_vencimento,dias_faltam_para_vencimento,intervalo,data_resposta,resposta_menos_abertura,vencimento_menos_resposta
0,1001,2025-02-19,False,2025-03-01,10,10,2025-02-27,8,2
1,1002,2025-02-19,False,2025-03-04,13,13,2025-03-02,11,2
2,1003,2025-02-19,False,2025-03-03,12,12,2025-03-01,10,2
3,1004,2025-02-19,True,2025-03-01,10,10,2025-02-21,2,8
4,1005,2025-02-19,False,2025-03-01,10,10,2025-02-26,7,3
...,...,...,...,...,...,...,...,...,...
1995,2996,2025-02-19,False,2025-03-04,13,13,2025-03-02,11,2
1996,2997,2025-02-19,False,2025-03-04,13,13,2025-03-01,10,3
1997,2998,2025-02-19,False,2025-03-02,11,11,2025-02-25,6,5
1998,2999,2025-02-19,False,2025-03-01,10,10,2025-02-21,2,8


In [550]:
df.to_csv('df.csv', index=False)